In [3]:
from typing import NotRequired
from typing_extensions import TypedDict
import uuid

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
import requests

# Define a TypedDict to represent the state
class State(TypedDict):
    url: str
    result: NotRequired[str]

In [ ]:
def call_api(state: State):
    """Example node that makes an API request."""
    # 单个url，不用task就可以
    result = requests.get(state['url']).text[:100]  # Side-effect  #
    return {
        "result": result
    }

builder = StateGraph(State)
builder.add_node("call_api", call_api)

builder.add_edge(START, "call_api")
builder.add_edge("call_api", END)

checkpointer = InMemorySaver()

graph = builder.compile(checkpointer=checkpointer)

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

graph.invoke({"url": "https://www.example.com"}, config)

{'url': 'https://www.example.com',
 'result': '<!doctype html><html lang="en"><head><title>Example Domain</title><meta name="viewport" content="wid'}

In [ ]:
from langgraph.func import task

@task
def _make_request(url: str):
    """Make a request."""
    return requests.get(url).text[:100]  

def call_api(state: State):
    """Example node that makes an API request."""
    # 多个url，拆成每个url一个小task，每个小task执行完毕会立即将结果持久化，如果恢复会从上次的地方继续运行
    requests = [_make_request(url) for url in state['urls']]  
    results = [request.result() for request in requests]
    return {
        "results": results
    }

builder = StateGraph(State)
builder.add_node("call_api", call_api)

builder.add_edge(START, "call_api")
builder.add_edge("call_api", END)

checkpointer = InMemorySaver()

graph = builder.compile(checkpointer=checkpointer)

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

graph.invoke({"urls": ["https://www.example.com"]}, config)